In [ ]:
!pip install easyocr


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.2 MB/s eta 0:00:00


In [ ]:
!pip install google-cloud-pubsub


In [ ]:
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import easyocr

In [ ]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
    # decode base64 image
    image_bytes = b64decode(js_reply.split(',')[1])
    # convert bytes to numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # decode numpy array into OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)

    return img

# function to convert EasyOCR result into bounding box image
def ocr_to_bbox_img(img, results):
    bbox_array = np.copy(img)
    for (bbox, text, prob) in results:
        (top_left, top_right, bottom_right, bottom_left) = bbox
        top_left = tuple(map(int, top_left))
        bottom_right = tuple(map(int, bottom_right))
        bbox_array = cv2.rectangle(bbox_array, top_left, bottom_right, (255, 0, 0), 2)
    return bbox_array

# initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };


      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});

//       stream = await navigator.mediaDevices.getUserMedia(
//         {video: {
//         facingMode: "environment",
//         frameRate: { ideal: 10, max: 15 },
//         width: { ideal: 640 },
//         height: { ideal: 480 }
//         mandatory: {
//             sourceUrl: 'http://192.168.100.73:8080/video',
//             sourceType: 'url',
//             minWidth: 640,
//             minHeight: 480,
//             maxWidth: 640,
//             maxHeight: 480
//         }
//     }
// });


      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
    display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

In [ ]:
pip install fuzzywuzzy

In [ ]:
from fuzzywuzzy import fuzz

# Threshold for fuzzy matching (adjust as needed)
fuzzy_threshold = 70

# Initialize an empty list to store detected text
detected_text_list = []

# Define the database of strings to match
database_strings = ["STL 57", "57 STL", "LEB 40", "40 LEB", "4o LEB", "LEB 4o", "ABC 123"]

# start streaming video from webcam
video_stream()
# label for video
label_html = 'Capturing...'
# initialize bounding box to empty
bbox = ''
count = 0

while True:
    try:
        js_reply = video_frame(label_html, bbox)
        if not js_reply:
            break

        # convert JS response to OpenCV Image
        img = js_to_image(js_reply["img"])

        if img is not None:
            # perform EasyOCR text detection
            results = reader.readtext(img)

            # Extract and store detected text
            detected_text = [result[1] for result in results]
            detected_text_list.extend(detected_text)

            # Check if any text is detected
            if detected_text:
            # Print the detected text for each frame
                print("Detected Text:", detected_text)

            # Check if the detected text matches or is similar to any string in the database
                x = 0  # match_status
                for text in detected_text:
                # Use fuzzy matching to handle slight variations
                    if any(fuzz.ratio(text, db_string) > fuzzy_threshold for db_string in database_strings):
                    # Set match_status to 'high' when there is a match or similarity
                        x = 1
                        break

            # Print the match status
                print(x)
                x = str(x)
                x = x.encode('utf-8')
                future = publisher.publish(topic_path, x)

                # Optional: Process further with the detected text and match status as needed

                # create transparent overlay for bounding box
                bbox_img = ocr_to_bbox_img(np.zeros_like(img), results)

                # convert overlay of bbox into bytes
                bbox_bytes = cv2.imencode('.jpg', bbox_img)[1].tobytes()
                bbox = 'data:image/jpeg;base64,{}'.format(str(b64encode(bbox_bytes), 'utf-8'))
        else:
            print("Error: Failed to decode image.")
    except Exception as e:
        print("Error:", str(e))

<IPython.core.display.Javascript object>

Detected Text: ['TL 57_']
1
Detected Text: ['STL', '57']
0
Detected Text: ['57', 'STL']
0
Detected Text: ['STL', '57']
0
Detected Text: ['57', 'STL']
0
Detected Text: ['57', 'STL']
0
Detected Text: ['57', 'STL']
0
Detected Text: ['ST 57']
1
Detected Text: ['ST 57']
1
Detected Text: ['STL']
0
Detected Text: ['STL 57']
1
Detected Text: ['STL', '57']
0
Detected Text: ['57', 'STL']
0
Detected Text: ['4 =', '=']
0
Detected Text: ['4e_123', '6ure']
0
Detected Text: ['6usb', 'Aec 12 3']
0
Detected Text: ['(use', 'ABc 12 3']
1
Detected Text: ['(usk', 'Aec 12 3']
0
Detected Text: ['a', '4-3']
0
Detected Text: ['4Bci?s', '2D']
0
Detected Text: ['ABc123', '@eb']
1
Detected Text: ['423', 'ABc', 'eb']
0
Detected Text: ['ABC/23', 'eb']
1
Detected Text: ['ABC123', 'eb']
1
Detected Text: ['ABC123', 'Zeb']
1
Detected Text: ['AB123', '@eb']
1
Detected Text: ['ABc123', '@eb']
1
Detected Text: ['ABC123', 'Beb']
1
Detected Text: ['ABc/23', 'Ruee']
0
Detected Text: ['ABC123', 'fuab']
1
Detected Text: ['ABC 

In [ ]:
import os
from google.cloud import pubsub_v1
credentials_path = '/content/arduino.privatekey.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path
publisher = pubsub_v1.PublisherClient()
topic_path = 'projects/arduino-409416/topics/arduinouno'

DONT RUN THE BELOW FILE, ITS ALREADY ADDED IN ABOVE

In [ ]:
x=8
x = str(x)
x = x.encode('utf-8')
future = publisher.publish(topic_path, x)